# **Tokenization**

## **TOC:**

- 1) **[Introduction](#intro)**

- 2) **[Character Tokenization](#chartoken)**

- 3) **[Word Tokenization](#wordtoken)**

- 4) **[Subword Tokenization](#subwordtoken)**

    - 4.1) **[Auto Tokenizer](#autotokenizer)**
    - 4.2) **[Specific Tokenizer](#specifictokenizer)**

- 5) **[Tokenizing the Dataset](#tokenizingdataset)**
    
    - 5.1) **[HuggingFace Dataset](#huggingdataset)**
    
    - 5.2) **[Custom Dataset](#customdataset)**

Transformers provides a convenient AutoTokenizer class that allows you to quickly load
the tokenizer associated with a pretrained model

<center><img src="figures/attention_masks.png" width=600></center>